In [ ]:
# Imports & Setup

import os

import json
import redis

import pickle
import hashlib

import numpy as np
import pandas as pd

import faiss

import google.generativeai as genai
from google.generativeai import embed_content

from typing import Dict, Any, Optional


In [ ]:
# Configuration 

API_KEY = 'your-api-key-here'

genai.configure(api_key=API_KEY)

GEN_MODEL = genai.GenerativeModel("gemini-2.5-flash")

EMBED_MODEL = "models/text-embedding-004"


In [3]:
# Paths

DIR_VECTORS = "../vectors"
DIR_PROCESSED = "../data/processed"

In [ ]:
# Redis Setup

redis_client = redis.Redis( 
                            host="localhost",
                            port=6379, db=0, 
                            decode_responses=False)

assert redis_client.ping(), "Redis is not running!"

In [ ]:
# Load Assets (Global State)

print("Loading System Assets")

# Load Vector Transforms & Indexes

pca = faiss.read_VectorTransform(f"{DIR_VECTORS}/pca_transform.faiss")

trades_index = faiss.read_index(f"{DIR_VECTORS}/trades_index/index_pq.faiss")

holdings_index = faiss.read_index(f"{DIR_VECTORS}/holdings_index/index_pq.faiss")



Loading System Assets


In [6]:
# Load Validated Data (For Context Mapping)

trades_data = pd.read_csv(f"{DIR_PROCESSED}/trades_validated.csv")

holdings_data = pd.read_csv(f"{DIR_PROCESSED}/holdings_validated.csv")


In [7]:
# Set Search Depth

trades_index.nprobe = 8

holdings_index.nprobe = 8

print("Assets Loaded.")

Assets Loaded.


In [ ]:
# Caching Logic

PIPELINE_VER = "v3"

def build_cache_key(prefix: str, payload: Dict[str, Any]) -> str:

    """Creates a deterministic hash for Redis keys."""

    raw = json.dumps(payload, sort_keys=True)

    digest = hashlib.sha256(raw.encode()).hexdigest()
    
    return f"{prefix}:{PIPELINE_VER}:{digest}"



In [9]:
def get_query_embedding(query: str) -> np.ndarray:

    """Checks Redis for embedding before calling API."""

    key = build_cache_key("query_embedding", {"query": query})
    
    # Cache Hit

    cached = redis_client.get(key)
    if cached: 
        return pickle.loads(cached)
    
    # Cache Miss

    res = embed_content(model=EMBED_MODEL, content=query)
    vec = np.array(res["embedding"], dtype="float32")
    
    # Store in Redis (24h expiry)
    
    redis_client.set(key, pickle.dumps(vec), ex=86400)
    return vec

In [ ]:
# Intent Classification

INTENT_PROMPT = """
                   
You are a STRICT intent classification engine for a financial analytics system.

IMPORTANT CONTEXT:
- The system has ONLY TWO datasets:
  1) Trades data (executed buy/sell transactions)
  2) Holdings data (current positions / portfolio holdings)
- You do NOT have access to the internet.
- You must NOT use any external or general knowledge.
- If the question cannot be answered using ONLY these datasets, it is UNSUPPORTED.

YOUR TASK:
Classify the user's query into EXACTLY ONE of the following intent labels.

INTENT DEFINITIONS:

TRADE_ONLY:
- Questions ONLY about executed trades or transactions
- Examples:
  - number of trades
  - buy/sell activity
  - transaction counts
  - trade history
  - trade performance

HOLDING_ONLY:
- Questions ONLY about current holdings or positions
- Examples:
  - current portfolio positions
  - holdings value
  - securities currently held
  - NAV / portfolio value (if derived from holdings)

MIXED:
- Questions that REQUIRE BOTH:
  - trade information AND
  - holding information
- Examples:
  - impact of trades on current holdings
  - how transactions changed positions
  - relationship between trades and holdings

UNSUPPORTED:
- Questions that CANNOT be answered from trades or holdings data
- Examples:
  - company executives
  - market news
  - predictions
  - external facts
  - vague or unrelated questions

USER QUERY:
"{query}"

OUTPUT RULES (VERY IMPORTANT):
- Output ONLY ONE label
- Output must be EXACTLY one of:
  TRADE_ONLY, HOLDING_ONLY, MIXED, UNSUPPORTED
- Do NOT explain your reasoning
- Do NOT add extra text
- Do NOT add punctuation
"""

def resolve_intent(query: str) -> Dict[str, Any]:

    """Determines which datasets to query."""

    q_lower = query.lower()
    
    # Fast Path (Rule-Based)

    if "trade" in q_lower and "holding" in q_lower:
        return {"intent": "MIXED", "use_trades": True, "use_holdings": True}
    
    # Slow Path (LLM)
    
    res = GEN_MODEL.generate_content(INTENT_PROMPT.format(query=query))
    intent = res.text.strip()
    
    return {
        "intent": intent,
        "use_trades": intent in ["TRADE_ONLY", "MIXED"],
        "use_holdings": intent in ["HOLDING_ONLY", "MIXED"]
    }




In [11]:
# Cell 5: Vector Retrieval
def retrieve_context(query: str, plan: Dict[str, Any]) -> Optional[str]:
    """Fetches relevant rows from CSVs based on vector similarity."""
    
    # 1. Embed & Reduce
    query_emb = get_query_embedding(query)
    query_pca = pca.apply_py(query_emb.reshape(1, -1))
    
    context_lines = []
    
    # 2. Search Trades
    if plan['use_trades']:
        distances, indices = trades_index.search(query_pca, k=5)
        for idx in indices[0]:
            if idx != -1:
                row_str = trades_data.iloc[idx].to_string()
                context_lines.append(f"TRADE DATA:\n{row_str}")

    # 3. Search Holdings
    if plan['use_holdings']:
        distances, indices = holdings_index.search(query_pca, k=5)
        for idx in indices[0]:
            if idx != -1:
                row_str = holdings_data.iloc[idx].to_string()
                context_lines.append(f"HOLDING DATA:\n{row_str}")
                
    return "\n---\n".join(context_lines) if context_lines else None

In [12]:
# Generation Layer

ANSWER_PROMPT = """
You are a financial assistant. Use ONLY the context below. 
If the answer is not in the context, say "Sorry can not find the answer".
Do not make assumptions.

Context:
{context}

Question: {question}
"""

def generate_answer(query: str) -> str:

    """Orchestrates Plan -> Retrieve -> Generate."""
    
    # Step 1: Intent

    plan = resolve_intent(query)

    if plan['intent'] == "UNSUPPORTED":
        return "Sorry can not find the answer"
        
    # Step 2: Retrieve

    context_text = retrieve_context(query, plan)

    if not context_text:
        return "Sorry can not find the answer"
        
    # Step 3: Generate

    prompt = ANSWER_PROMPT.format(context=context_text, question=query)

    try:
        response = GEN_MODEL.generate_content(prompt)
        return response.text.strip()
    
    except Exception:
        return "Sorry can not find the answer"




In [ ]:
# Interactive Loop
print("Financial Engine Ready. Type 'exit' to quit.")

while True:
    user_input = input("\nEnter Query: ")
    if user_input.lower() == 'exit':
        break
        
    answer = generate_answer(user_input)
    print(f"Answer: {answer}")

🤖 Financial Engine Ready. Type 'exit' to quit.
Answer: Sorry can not find the answer
